In [1]:
from llm_client.prompt_builder import  PromptBuilder
import pandas as pd
from config.settings import Settings
from content_retrieval import llm,doc_store
from pipelines.processors import summarize_new_documents, add_new_documents_to_docstore
settings = Settings()

Initialized new FAISS index with dimension 3072.
Syncing VectorStore with DocumentStore...
VectorStore is already in sync. No new documents to add.


In [2]:
kme_vertaaltabel = pd.read_csv(settings.data_root /  "kme_vertaaltabel.csv",sep=';').set_index("KME_ID")

# Opbouwen Doc store van pdf

In [3]:
kme_documents = pd.read_pickle(settings.content_folder / "extracted_df_km.pkl").reset_index()
kme_documents = kme_documents.set_index("km_nummer").join(kme_vertaaltabel).reset_index()
ob_documents = kme_documents.query("BELASTINGSOORT == 'OB - Omzetbelasting'")
results = add_new_documents_to_docstore(ob_documents,doc_store)

In [4]:
summary_processor = PromptBuilder(template_path='prompt_templates',name='summarize')

### Samenvatten en segmenteren content

In [6]:
stats = summarize_new_documents(
    doc_store=doc_store,
    prompt_builder=summary_processor,
    llm=llm,
    max_workers=10,
    start=0,
    count=1500,
    show_progress=True
)

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

subscription_id = "f475c0e3-1450-42c9-a3b7-cbc1e54c2078"
resource_group = "ino-rg-kis-poc"
workspace_name = "KIS-POC"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

# --- Get the MLflow tracking URI ---
mlflow_tracking_uri = ml_client.workspaces.get(workspace_name).mlflow_tracking_uri
print(f"MLflow Tracking URI: {mlflow_tracking_uri}")
mlflow.set_tracking_uri(mlflow_tracking_uri)

# --- Set the tracking URI for your MLflow environment ---
import mlflow



Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed


Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


MLflow Tracking URI: azureml://northeurope.api.azureml.ms/mlflow/v2.0/subscriptions/f475c0e3-1450-42c9-a3b7-cbc1e54c2078/resourceGroups/ino-rg-kis-poc/providers/Microsoft.MachineLearningServices/workspaces/KIS-POC


In [10]:
ml_client.workspaces.get("KIS-POC").mlflow_tracking_uri

HttpResponseError: (InvalidSubscriptionId) The provided subscription identifier 'None' is malformed or invalid.
Code: InvalidSubscriptionId
Message: The provided subscription identifier 'None' is malformed or invalid.